In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
from placeholder_events import *
import requests

In [2]:
#function to create api connection to google sheets
def connect_to_gs(service_account_key):
    scopes = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = service_account.Credentials.from_service_account_info(service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection


#create connection
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
#function to write data to google sheets
def write_google_sheets_data(gc, df, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        df_values = df.values.tolist()
        gs.values_append(sheet_name, {'valueInputOption': 'RAW'}, {
                    'values': df_values})

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
     

In [4]:
#function to fetch data from google sheets
def fetch_google_sheets_data(gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    
#fetch data


In [5]:
def fetch_max_gw(_gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # Convert the first column to numeric
        df[headers[0]] = pd.to_numeric(df[headers[0]])

        # Get the maximum value of the first column
        max_value = df[headers[0]].max()

        return int(max_value)

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [6]:
def update(_gc):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()

    events = pd.DataFrame(general_response['events'])[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]
    if len(events[events['is_current'] == True]) == 0:
        print('season hasn\'t started pull placeholder data')
        events = pd.DataFrame(placeholder_events)[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]

    
    gw = int(events[events["is_current"] == True]["id"])
    finished = bool(events[events["is_current"] == True]["finished"].values)
    data_checked = bool(events[events["is_current"] == True]["data_checked"].values)
    
    if finished and data_checked:
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        max_stored_gw = fetch_max_gw(_gc, gameweek_results_table, google_sheet_key)
        if max_stored_gw < gw:      
            print('max stored_gw < current week')
            print('we will update here')
            return True
        
        else: 
            print('data is already latest - pull from gs')
            return False
    else:
        print("its either a new week or the old week hasnt completetly finished - pull from gs")
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        return False

In [ ]:
def gameweek_results_update():
    #pull results from fpl API 
    #append them to gs

    #calc first and last place for the week
    #return first and last and current_week
    return None

In [ ]:
#takes current week from gameweek_results_update

#cache this on indefinitely and then reset cache when update is true

def auto_drinks_update(current_week, last):
    #loop through teams and create df fines like:

    #event: current_week
    #nominator_name = 'auto'
    #drinker_name = 'person fined'
    #drink_type: red card/own goal/negative pts/last
    #nomination_created_date = now()
    #nomination_deadline_date = now() + 7 days
    #quantity =  1
    return None


In [ ]:
def managers_update():
    #check league managers
    #check gs managers
    #if difference append with event_joined = current_gw and uno_reverse =yes
    return None

In [ ]:
#this function is only used to store variables that don't change betweem gameweeks, it will only reset when theres an update

def cached_variables(first_place, last_place, redcards, own_goals, negative_points, current_week):
    return first_place, last_place, redcards, own_goals, negative_points, current_week

In [7]:
#determine if an update is needed every 6 hours
if update(gs_connection) == True:
    print('There is new data, google sheets will be updated now')
    # gameweek_results_update()
    # auto_drinks_update(current_week, last)
    # managers_update()

    # cached_variables.clear()
    # first_place, last_place, redcards, own_goals, negative_points, current_week = cached_variables(first_place, last_place, redcards, own_goals, negative_points, current_week)
else:
    print('Data in google sheets is up to date')
    #create placeholders
    first_place = 'Connor McDonald'
    last_place = 'Alex Wietzorrek'
    redcards = []
    own_goals = []
    negative_points = []
    current_week = 0

#we will pull from gs here regardless - hopefully the condition above handles the state of this data

season hasn't started pull placeholder data
its either a new week or the old week hasnt completetly finished - pull from gs
finished: False
checked: False
game week: 1
do nothing
